In [1]:
# Import libraries
## for profiling
from profiler.main import profile

# for titiler-xarray
from typing import Any
from zarr_reader import xarray_open_dataset, get_variable
import morecantile
from xarray_tile_reader import XarrayTileReader

# for titiler-pgstac
from titiler.pgstac.mosaic import PGSTACBackend
from typing import Any, Dict, List, Tuple
from rio_tiler.mosaic import mosaic_reader
from rio_tiler.models import ImageData
from geojson_pydantic import Polygon

# Only if wanting to see a picture
import io
from PIL import Image

In [2]:
import os
os.environ['AWS_PROFILE'] = 'veda-dev'
os.environ['CPL_TIMESTAMP'] = 'ON'
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR'

In [3]:
xyz_tile = (0,0,0)

## Profile titiler-pgstac

In [9]:
%load_ext autoreload
%autoreload
import pgstac.profile_pgstac as profile_pgstac 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
query = {
  "collections": [
    "CMIP6_ensemble_median_TAS"
  ],
  "filter": {
    "op": "t_intersects",
    "args": [
      {
        "property": "datetime"
      },
      {
        "interval": [
          "1950-04-01T00:00:00Z",
          #"1950-05-01T00:00:00Z"
        ]
      }
    ]
  },
  "filter-lang": "cql2-json"
}

image_and_assets, logs = profile_pgstac.tile(*xyz_tile, query=dict(query))
logs

src_dst is CustomSTACReader(bounds=[-180.0, -60.0, 180.0, 90.0], crs=CRS.from_epsg(4326), geographic_crs=CRS.from_epsg(4326), assets=['data'], input={'id': 'tas_month_ensemble-median_historical_195004.tif', 'bbox': [-180.0, -60.0, 180.0, 90.0], 'assets': {'data': {'href': 's3://climatedashboard-data/cmip6/monthly/CMIP6_ensemble_median/tas/tas_month_ensemble-median_historical_195004.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'raster:bands': [{'scale': 1.0, 'nodata': 'nan', 'offset': 0.0, 'sampling': 'area', 'data_type': 'float32', 'histogram': {'max': 307.9095153808594, 'min': 235.8828125, 'count': 11, 'buckets': [765, 4692, 9726, 10226, 14887, 17829, 17271, 14494, 32746, 11563]}, 'statistics': {'mean': 281.1649565197952, 'stddev': 16.612070663453505, 'maximum': 307.9095153808594, 'minimum': 235.8828125, 'valid_percent': 30.691735582552692}}]}}, 'collection': 'CMIP6_ensemble_median_TAS'}, tms=<TileMatrixSet title='Google Maps Compatible 

{'cprofile': ['   ncalls  tottime  percall  cumtime  percall filename:lineno(function)',
  '        4    0.111    0.028    0.112    0.028 connection.py:949(wait)',
  "        9    0.054    0.006    0.054    0.006 {method 'acquire' of '_thread.lock' objects}",
  '        1    0.001    0.001    0.172    0.172 profile_pgstac.py:130(tile)',
  '        2    0.001    0.000    0.001    0.000 core.py:3433(__setmask__)']}

## Profile titiler-xarray

In [ ]:
@profile(add_to_return=True, quiet=False, log_library='s3fs')
def tile(src_path: str, x: int, y: int, z: int, *, variable: str, time_slice: str = None, **kwargs: Any):

    with xarray_open_dataset(
        src_path,
        group=z,
        decode_times=False,
        **kwargs,
    ) as dataset:
        dataarray = get_variable(dataset, variable=variable, time_slice=time_slice)
        
        with XarrayTileReader(dataarray) as src_dst:
            return src_dst.tile(x, y, z)

In [ ]:
image_and_assets, logs = tile(
    "s3://power-analysis-ready-datastore/power_901_monthly_meteorology_utc.zarr", 
    *xyz_tile,
    reference=False,
    variable="TS",
)
# logs

In [ ]:
content = image_and_assets.render(
    img_format='PNG'
)

im = Image.open(io.BytesIO(content))
im

In [ ]:
content = image_and_assets[0].render(
    img_format='PNG'
)

im = Image.open(io.BytesIO(content))
im